# Predicting fragment ion intensity from peptide sequence

In this vignette, we build Transformer model to predict the intensity of b- and y-series fragment ions for a peptide from its sequence and charge state, similar to Prosit, MS2PIP, and El Fragmentador. 
We'll use a subset the same ProteomeTools training, validation, and test data from [the original Prosit paper](https://www.nature.com/articles/s41592-019-0426-7).
The data provided by Gessulat, et al is already preprocessed and in an HDF5 file format, which means that this notebook does not contain the code needed to generate annotations for training from arbitrary mass spectrometry data files.
For that, we recommend looking at [this ProteomicsML tutorial](https://proteomicsml.org/tutorials/fragmentation/raw-to-prosit.html).

**Before proceeding with this notebook, make sure to switch a GPU runtime on Google Colab.** To do this, click `Runtime` -> `Change runtime type`, and select `GPU` under `Hardware accelerator`. A new `GPU type` box should appear below. While any will work, we used the `T4` GPU to run this notebook previously.

## Setup

The follow code installs the additional dependencies we'll need: Depthcharge, PyTorch Lightning, and Tensorboard. 
It also downloads the data that we'll be using from [FigShare](https://figshare.com/projects/prosit/35582). 
Please not that this data is several GB in size, so it may take a few minutes to download.
In the end, we are left with our data in three HDF5 files in our working directory: `train.hdf5`, `valid.hdf5`, and `test.hdf5`.

In [ ]:
%%capture
%%bash
pip install lightning tensorboard depthcharge-ms
for FILE in "train.hdf5 24635459" "valid.hdf5 24635442" "test.hdf5 24635438"
do
    set -- $FILE
    if [ ! -f $1 ]; then
        wget -nc https://figshare.com/ndownloader/files/$2
        mv $2 $1
    fi
done

## Import the libraries we'll need
To work with our data, we'll need a handful of standard data science tools (NumPy, Pandas, etc.).
For model building, we'll use PyTorch Lightning to wrap our model from Depthcharge, making it easy to train.

From Depthcharge, we'll use the following classes:
- `PeptideDataset` - This is a PyTorch Dataset that is designed to hold peptide sequences, their charge states, and additional metadata.
- `FeedForward` - This is a utility PyTorch Module for quickly creating feed forward neural networks.
- `PeptideTokenizer` - This class defines the amino acid alphabet, including modifications, that are valid tokens for our model. 
  It also tells Depthcharge how to convert a peptide sequence into tokens and back. 
  First-class support for ProForma formatted peptide sequences is included out-of-the-box.
- `PeptideTransformerEncoder` - This is a PyTorch Module that embeds the peptide and its residues using a Transformer architecture.
- `FloatEncoder` - This PyTorch Module encodes floating point numbers with sinusoidal waveforms. We use it here to encode the normalized collision energe (NCE)>

After importing these libraries, the following code also sets a plotting theme and a random seed for reproducibility.

In [ ]:
import os

import einops
import h5py
import lightning.pytorch as pl
import matplotlib.pyplot as plt
import numba as nb
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from depthcharge.data import PeptideDataset
from depthcharge.encoders import FloatEncoder
from depthcharge.feedforward import FeedForward
from depthcharge.tokenizers import PeptideTokenizer
from depthcharge.transformers import PeptideTransformerEncoder

# Set our plotting theme:
sns.set_style("ticks")

# Set random seeds
pl.seed_everything(42, workers=True)

## Create a tokenizer
Now that we know the peptides that we want to consider, we need to create a tokenizer that accounts for all of the amino acids and modifications that may be present. Fortunately, the `PeptideTokenizer` class has a `from_proforma()` method that allows us to extract the amino acids and modifications from a collection of peptides.

In [ ]:
tokenizer = PeptideTokenizer.from_proforma(
    sequences="ACDEFGHIKLMNPQRSTVWYM[Oxidation]", 
    replace_isoleucine_with_leucine=False, 
    reverse=False,
)

pd.DataFrame(tokenizer.residues.items(), columns=["Token", "Mass"])

It looks like our tokenizer has captured all of the residues we expect.

## Preparing Datasets
Now we need to prepare our PyTorch `Dataset`s and their corresponding PyTorch `DataLoader`s. 
Here, we use Depthcharge's `PeptideDataset` class, which handles transforming the peptide strings into PyTorch tensors for modeling. 
However, it takes some work to parse the data in the Prosit HDF5 file format, extract peptide sequence, and arrange the measured intensities in manner that is conducive for modeling with a Transformer.

Our solution was to create a `PrositDataset` class, that is a subclass Depthcharge's `PeptideDataset` class.
This class takes care of all these parsing and preprocessing steps.
Additionally, this data is quite large and unable to fit in memory on our standard Google Colab instance. 
Although we could write a code to extract specific peptides from the HDF5 file dynamically, for the sake of brevity and speed we decided to extract a subset of each of the training, validation, and test datasets.
We chose to subset each dataset by extracting every $k$th peptide, because the peptides are sorted lexographically within each file, and generating a truly random sample for this dataset is difficult.
The `PrositDataset` chooses $k$ to yield approximately number of peptides requested.

Running the code below may take a few minutes, but will yield the training, validation, and test data loaders that we need train and evaluate our model.

In [ ]:
@nb.njit
def vecs2seqs(vecs, alphabet):
    """Convert Prosit vectors to peptide sequenes"""
    for idx, seq_idx in enumerate(vecs):
        yield "".join([alphabet[i - 1] for i in seq_idx if i])

@nb.njit
def flip_and_shift_b_ions(ions, n_ions):
    """Flip the b_ions and shift them one down.

    This let's them match the order of our Transformer model.
    """
    for idx, n_ion in enumerate(n_ions): 
        ions[idx, 1:n_ion+1, 1, :] = ions[idx, n_ion-1::-1, 1, :]

    return ions


class PrositDataset(PeptideDataset):
    """A class for the Prosit HDF5 files."""
    def __init__(self, tokenizer, hdf5_file, max_examples=1_000_000):
        """Initialize the Prosit Dataset"""
        alphabet = list(tokenizer.residues.keys())
        with h5py.File(hdf5_file) as data:
            n_rows = data["scan_number"].shape[0]

        if n_rows > max_examples:
            # The peptides are lexigraphically sorted, so we'll take a 
            # diverse subset with creative indexing.
            print(f"  -> Found {n_rows} peptides. Subsetting to ~{max_examples}...")
            step = int(np.floor(n_rows / max_examples))
        else:
            step = 1

        print("  -> Reading from HDF5 file....")
        with h5py.File(hdf5_file) as data:
            charge = np.argmax(data["precursor_charge_onehot"][::step], axis=1) + 1
            charge = torch.tensor(charge).to("cuda")
            nce = torch.tensor(data["collision_energy_aligned_normed"][::step]).to("cuda")
            seq = vecs2seqs(data["sequence_integer"][::step], np.array(alphabet))
            intensities = data["intensities_raw"][::step]
            n_rows = len(charge)

        print("  -> Preprocessing intensities...")
        # Transform the intensities for our Transformer.
        # Intensities are shape (L, I, Z) where:
        # L = The peptide length - 1, ordered from lowest mass to highest.
        # I = The ion series, (y, b)
        # Z = The charge state (increasing)
        intensities = intensities.reshape([n_rows, 29, 2, 3])
        n_ions = (intensities[:, :, 0, 0] >= 0).sum(axis=1)
        
        # Need an extra space because we want to shift b ions.
        intensities = np.pad(
            intensities,
            ((0, 0), (0, 1), (0, 0), (0, 0)), 
            "constant", 
            constant_values=-1,
        )

        intensities = flip_and_shift_b_ions(intensities, n_ions)
        intensities[:, 0, 1, :] = -1
        intensities = torch.tensor(intensities).to("cuda")

        print("  -> Tokenizing peptides...")
        super().__init__(tokenizer, seq, charge, nce, intensities)


print("Loading the training dataset...")
train_dataset = PrositDataset(tokenizer, "train.hdf5", 200_000)
print("Loading the validation dataset...")
validation_dataset = PrositDataset(tokenizer, "valid.hdf5", 100_000)
print("Loading the test dataset...")
test_dataset = PrositDataset(tokenizer, "test.hdf5", 100_000)

# The GPU memory on this instance is larger than the host, so
# we put data on the gpu to run fast.
for dset in (train_dataset, validation_dataset, test_dataset):
    tensors = []
    for data in dset.tensors:
        tensors.append(data.to("cuda"))

    dset.tensors = tuple(tensors)

train_loader = train_dataset.loader(
    batch_size=128, shuffle=True,
)
validation_loader = validation_dataset.loader(
    batch_size=1024, shuffle=False,
)

test_loader = test_dataset.loader(
    batch_size=1024, shuffle=False,
)

## Create a model

Time to create a deep learning model using PyTorch Lightning and Depthcharge! 
Our model consists of a `PeptideTransformerEncoder` module to embed peptides and a `FeedForward` module to predict the intensity for charge states 1-3 of each expected b and y ion.
With PyTorch Lightning, we need to specify the modules that comprise our model, define the optimizer(s) we will use to train it, and tell Lightning how to run the model when training, validating, and predicting.

For this task, we're trying to minimize the spectral angle distance loss function:

$$  L = \frac{2}{n\pi}\sum^{n}_{i=1}\cos^{-1}\left( \frac{Y_i \cdot \hat{Y}_i}{||Y_i|| \cdot ||\hat{Y}_i||}\right)  $$

Where $n$ is the number of peptides, $Y_i$ is a vector of measured intensities, and $\hat{Y}_i$ is the vector of predicted intensities.

In [ ]:
def masked_spectral_angle(true, pred):
    """This is an PyTorch adaptation of the Prosit implementation here:
    https://github.com/kusterlab/prosit/blob/dd16c47f8c3f4cfbd7ae84a1ca92a4d117e5e9ec/prosit/losses.py#L4-L16
    """
    true = true.flatten(start_dim=1)
    pred = pred.flatten(start_dim=1)
    epsilon = torch.finfo(torch.float32).eps
    pred_masked = ((true + 1) * pred) / (true + 1 + epsilon)
    true_masked = ((true + 1) * true) / (true + 1 + epsilon)
    pred_norm = F.normalize(true_masked, p=2, dim=-1)
    true_norm = F.normalize(pred_masked, p=2, dim=-1)
    product = torch.sum(pred_norm * true_norm, dim=1)
    arccos = torch.acos(product)
    return 2 * arccos / np.pi


class FragmentPredictor(pl.LightningModule):
    """A Transformer model for CCS prediction"""
    def __init__(self, tokenizer, d_model, n_layers):
        """Initialize the CCSPredictor"""
        super().__init__()
        self.peptide_encoder = PeptideTransformerEncoder(
            n_tokens=tokenizer,
            d_model=d_model,
            n_layers=n_layers,
            max_charge=10,
        )
        self.nce_encoder = FloatEncoder(d_model, max_wavelength=1)
        self.fragment_head = FeedForward(d_model, 6, 3)

    def step(self, batch, batch_idx):
        """A training/validation/inference step."""
        seqs, charges, nce, intensities = batch
        embedded, mask = self.peptide_encoder(seqs, charges)
        emb_nce = self.nce_encoder(nce[:, None])
        pred = self.fragment_head(embedded + emb_nce) 

        # Reshape for the Prosit data:
        pred = einops.rearrange(pred, "B L (I Z) -> B I Z L", I=2)
        pred = F.pad(pred, (0, 30 - pred.shape[-1]), "constant", 0)
        pred = einops.rearrange(pred, "B I Z L -> B L I Z")

        # Calculate the loss
        if intensities is not None:
            intensities = intensities.type_as(pred)
            loss = masked_spectral_angle(intensities, pred).mean()
        else:
            loss = None

        return pred, loss

    def training_step(self, batch, batch_idx):
        """A training step"""
        _, loss = self.step(batch, batch_idx)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """A validation step"""
        _, loss = self.step(batch, batch_idx)
        self.log("validation_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def predict_step(self, batch, batch_idx):
        """An inference step"""
        pred, _ = self.step(batch, batch_idx)
        return pred

    def configure_optimizers(self):
        """Configure the optimizer for training."""
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

## Optional: Setup Tensorboard

Tensorboard is a tool used to track the training progress of deep learning models in real time. 
Running the cell below will start Tensorboard and tell it to listen to the logs that will be written by our model.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

## Train the model

With our model defined and our data loaders ready to go, its time to fit the model to the data.
The PyTorch Lightning `Trainer` will handle a lot of the training for us. 
We enable an early stopping criterium here, so that the trainer will stop once the loss on our validation dataset stops improving. 
This model should take <2 hours to train.
If you've enabled Tensorboard in the previous cell, scroll back up while the model trains and you'll be able to watch its progress.

In [ ]:
# Create a model:
model = FragmentPredictor(tokenizer, d_model=64, n_layers=6)

# Create the trainer:
early_stopping = EarlyStopping(monitor="validation_loss", patience=3)
trainer = pl.Trainer(
    callbacks=[early_stopping],
    max_epochs=10, 
)

# Train the model:
trainer.fit(
    model=model, 
    train_dataloaders=train_loader, 
    val_dataloaders=validation_loader,
)

## Predict on the Validation dataset

We now want to see how we've done, aside from just looking at the MSE. 
To get the predicte CCS for every peptide in our validation set, we use the `predict()` method for the trainer on our validation data loader.
We then visualize our distribution of spectral angle distances using the empirical cumulative density function (ECDF).

In [ ]:
preds = torch.cat(trainer.predict(model, validation_loader)).cpu()
sa_distance = masked_spectral_angle(validation_dataset.tensors[3].cpu(), preds).detach().numpy()

plt.figure()
sns.ecdfplot(x=(1 - sa_distance), stat="count")
plt.xlabel("1 - Spectral Angle")
plt.ylabel("Number of Peptides")
plt.show()

## Predict on the Test dataset

Like with our validation data, we use the `predict()` method to get the predicted CCS for each of our test dataset peptides. 
We make a ECDF plot and save the predictions to a file, which we used to make the visualizations on our poster.

In [ ]:
preds = torch.cat(trainer.predict(model, test_loader)).cpu()
sa_distance = masked_spectral_angle(test_dataset.tensors[3].cpu(), preds).detach().numpy()

plt.figure()
sns.ecdfplot(x=(1 - sa_distance), stat="count")
plt.xlabel("1 - Spectral Angle")
plt.ylabel("Number of Peptides")
plt.show()

torch.save(preds.detach().cpu(), "test.pt")

Awesome!

If you want to fully reproduce our figures from the poster, you'll need to clone [our GitHub repo](https://github.com/wfondrie/2023_asms-depthcharge), follow the instructions in the README for setting up your environment, and execute [intensity-prediction-figures.ipynb](https://github.com/wfondrie/2023_asms-depthcharge/blob/main/notebooks/intensity-prediction-figures.ipynb) Jupyter notebook.